In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.decomposition import TruncatedSVD
# If nltk stop word is not downloaded
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hbhak\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [8]:
# List of documents
a1 = "my name is Harsh."
a2 = "Harsh is too lazy."
a3 = "He is too cool."
a4 = "He is studying NLP."
a5 = "He has brown eyes."

df = pd.DataFrame()
df["documents"] = [a1,a2,a3,a4,a5]
df.head()

,documents
0,my name is Harsh.
1,Harsh is too lazy.
2,He is too cool.
3,He is studying NLP.
4,He has brown eyes.


In [9]:
# Preprocessing
df['clean_documents'] = df['documents'].str.replace("[^a-zA-Z#]", " ")
df['clean_documents'] = df['clean_documents'].fillna('').apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
df['clean_documents'] = df['clean_documents'].fillna('').apply(lambda x: x.lower())

df.head()

,documents,clean_documents
0,my name is Harsh.,name harsh
1,Harsh is too lazy.,harsh too lazy
2,He is too cool.,too cool
3,He is studying NLP.,studying nlp
4,He has brown eyes.,has brown eyes


In [10]:
import nltk 
nltk.download('stopwords') 
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hbhak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# tokenization
tokenized_doc = df['clean_documents'].fillna('').apply(lambda x: x.split())

# remove stop-words

#tokenized_doc = tokenized_doc.apply(lambda x: [itemstop_words = stopwords.words('english') for item in x if item not in stop_words])
# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words]) 
# de-tokenization
detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

df['clean_documents'] = detokenized_doc
df.head()

,documents,clean_documents
0,my name is Harsh.,name harsh
1,Harsh is too lazy.,harsh lazy
2,He is too cool.,cool
3,He is studying NLP.,studying nlp
4,He has brown eyes.,brown eyes


In [12]:
# TF-IDF vector
vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
X = vectorizer.fit_transform(df['clean_documents'])
X.toarray()

array([[0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.62791376, 0.77828292,
        0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.70710678, 0.70710678],
       [0.70710678, 0.        , 0.70710678, 0.        , 0.        ,
        0.        , 0.        ]])

In [13]:
X.shape 
#A56   U(5,5). S()

(5, 7)

In [14]:
# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)
lsa = svd_model.fit_transform(X)

In [15]:
#Documents - Topic vector
pd.options.display.float_format = '{:,.16f}'.format
topic_encoded_df = pd.DataFrame(lsa, columns = ["topic_1", "topic_2"])
topic_encoded_df["documents"] = df['clean_documents']
display(topic_encoded_df[["documents", "topic_1", "topic_2"]])

,documents,topic_1,topic_2
0,name harsh,0.9021955890080026,0.0000000000000000
1,harsh lazy,0.9021955890080025,-0.0000000000000000
2,cool,0.0000000000000000,-0.0000000000000000
3,studying nlp,0.0000000000000000,1.0000000000000000
4,brown eyes,0.0000000000000001,-0.0000000000000000


In [16]:
# Features or words used as features 
dictionary = vectorizer.get_feature_names()
dictionary

['brown', 'cool', 'eyes', 'harsh', 'lazy', 'nlp', 'studying']

In [18]:
# Term-Topic matrix
encoding_matrix = pd.DataFrame(svd_model.components_, index = ["topic_1","topic_2"], columns = (dictionary)).T
encoding_matrix

,topic_1,topic_2
brown,0.0000000000000001,0.0000000000000000
cool,-0.0000000000000000,0.0000000000000001
eyes,0.0000000000000001,0.0000000000000000
harsh,0.9021955890080026,0.0000000000000000
lazy,0.4313271602559978,-0.0000000000000001
nlp,0.0000000000000000,0.7071067811865476
studying,0.0000000000000000,0.7071067811865476
